In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.spatial
import matplotlib.pyplot as plt


# from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import pandas as pd
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
import torch
import pandas as pd
from pprint import pprint

import sys
import os
import glob


# Dataset

In [4]:
PATH = '../input/dataset-haick/train.json'

In [5]:
train_df = pd.read_json(PATH)

In [6]:
questions = []
for ind in range(8000):
    questions.extend(train_df.loc[train_df[f'question {ind}'].notna(), f'question {ind}'].tolist())
dataset = train_df.copy()
dataset['question 0'] = questions
dataset.rename(columns={'question 0': 'question'}, inplace=True)
dataset.drop(columns=[f'question {ind}' for ind in range(1, 8000)], inplace=True)

In [7]:
dataset.drop(columns = ['explanation' , 'category'])

,question,options,label
0,What is the measurement model for an interfero...,[The sampled antenna signals are modeled as th...,1
1,What is the method used by switches in IEEE 80...,"[Multiple antennas, Flexible deployment, Chann...",4
2,Why is accurate modeling of power consumption ...,[To accurately determine the number of bits th...,2
3,What happens to the current in an inductive ci...,"[The current drops to zero immediately, The cu...",4
4,Which frequency bands are massive MIMO systems...,"[Centimeter-wave bands, Millimeter-wave bands,...",2
...,...,...,...
7995,What does the capacitor do in an RF (radio fre...,"[Converts RF waves into DC signal, Observes an...",4
7996,What is the modulation technology called in Gi...,"[3D-PAM4, 4D-PAM5, 5D-PAM8, 6D-PAM16]",2
7997,What is the goal of vector quantization?,"[To minimize the MSE of the quantization, To r...",3
7998,Which interface is not required in the roaming...,"[N26 interface, S9 interface, N9 interface, S1...",2


In [8]:
def select_correct_answer(row):
    
    options_list = row['options']
    label = row['label'] - 1  # Adjust for 0-based indexing if labels start from 1
    correct_answer = options_list[label] if 0 <= label < len(options_list) else np.nan
    return pd.Series({'question': row['question'], 'options': options_list, 'correct_answer': correct_answer})

dataset = dataset.apply(select_correct_answer, axis=1)

In [9]:
dataset

,question,options,correct_answer
0,What is the measurement model for an interfero...,[The sampled antenna signals are modeled as th...,The sampled antenna signals are modeled as the...
1,What is the method used by switches in IEEE 80...,"[Multiple antennas, Flexible deployment, Chann...",Scheduled data transmission
2,Why is accurate modeling of power consumption ...,[To accurately determine the number of bits th...,To identify the non-negligible components of p...
3,What happens to the current in an inductive ci...,"[The current drops to zero immediately, The cu...",The current gradually decreases to zero
4,Which frequency bands are massive MIMO systems...,"[Centimeter-wave bands, Millimeter-wave bands,...",Millimeter-wave bands
...,...,...,...
7995,What does the capacitor do in an RF (radio fre...,"[Converts RF waves into DC signal, Observes an...",Ensures a smooth delivery of power to the load
7996,What is the modulation technology called in Gi...,"[3D-PAM4, 4D-PAM5, 5D-PAM8, 6D-PAM16]",4D-PAM5
7997,What is the goal of vector quantization?,"[To minimize the MSE of the quantization, To r...",To minimize the distortion in encoding the source
7998,Which interface is not required in the roaming...,"[N26 interface, S9 interface, N9 interface, S1...",S9 interface


In [10]:
dataset.shape

(8000, 3)

In [12]:
df = dataset.copy()
df.head()

,question,options,correct_answer
0,What is the measurement model for an interfero...,[The sampled antenna signals are modeled as th...,The sampled antenna signals are modeled as the...
1,What is the method used by switches in IEEE 80...,"[Multiple antennas, Flexible deployment, Chann...",Scheduled data transmission
2,Why is accurate modeling of power consumption ...,[To accurately determine the number of bits th...,To identify the non-negligible components of p...
3,What happens to the current in an inductive ci...,"[The current drops to zero immediately, The cu...",The current gradually decreases to zero
4,Which frequency bands are massive MIMO systems...,"[Centimeter-wave bands, Millimeter-wave bands,...",Millimeter-wave bands


In [13]:
q_list = "question: " + df['question'] 
o_list = df['options']
a_list = df['correct_answer'] 

dict_data = {'source_text': q_list,
      'options': o_list,    
      'target_text': a_list}

df = pd.DataFrame(dict_data)
df.head()      

,source_text,options,target_text
0,question: What is the measurement model for an...,[The sampled antenna signals are modeled as th...,The sampled antenna signals are modeled as the...
1,question: What is the method used by switches ...,"[Multiple antennas, Flexible deployment, Chann...",Scheduled data transmission
2,question: Why is accurate modeling of power co...,[To accurately determine the number of bits th...,To identify the non-negligible components of p...
3,question: What happens to the current in an in...,"[The current drops to zero immediately, The cu...",The current gradually decreases to zero
4,question: Which frequency bands are massive MI...,"[Centimeter-wave bands, Millimeter-wave bands,...",Millimeter-wave bands


In [14]:
df['source_text'] = df.apply(lambda row: f"question: {row['source_text']} options: {' | '.join(row['options'])}", axis=1)

In [15]:
"""
df['options'] = df['options'].apply(tuple)

# Calculate the shape and unique counts
df_shape = df.shape
unique_source_texts = len(df['source_text'].unique())
unique_options = len(df['options'].unique())
unique_target_texts = len(df['target_text'].unique())

# Print the results
print(df_shape, unique_source_texts, unique_options, unique_target_texts)
"""

"\ndf['options'] = df['options'].apply(tuple)\n\n# Calculate the shape and unique counts\ndf_shape = df.shape\nunique_source_texts = len(df['source_text'].unique())\nunique_options = len(df['options'].unique())\nunique_target_texts = len(df['target_text'].unique())\n\n# Print the results\nprint(df_shape, unique_source_texts, unique_options, unique_target_texts)\n"

In [16]:
# splitting data into train and test data
train_data, val_data = train_test_split(df[:-100], test_size=0.2)
test_data = df[-100:]
train_data.shape, val_data.shape, test_data.shape

((6320, 3), (1580, 3), (100, 3))

# Model

In [ ]:
!pip install --upgrade simplet5

In [17]:
%%time
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df = train_data,
            eval_df = val_data, 
            source_max_token_len=256, 
            target_max_token_len=64, 
            batch_size=16, max_epochs=3, use_gpu=True)

Validation sanity check: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

CPU times: user 17min 45s, sys: 1min, total: 18min 46s
Wall time: 18min 54s


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [27]:
ls ./outputs

simplet5-epoch-0-train-loss-0.2288-val-loss-0.1641/
simplet5-epoch-0-train-loss-0.2585-val-loss-0.1934/
simplet5-epoch-1-train-loss-0.1519-val-loss-0.1695/
simplet5-epoch-1-train-loss-0.1655-val-loss-0.1928/
simplet5-epoch-2-train-loss-0.1103-val-loss-0.1781/
simplet5-epoch-2-train-loss-0.1189-val-loss-0.1992/
simplet5-epoch-3-train-loss-0.0813-val-loss-0.1911/


for running well, you should take the path of the last epoch .. for ex here "SimpleT5-epoch-2-train-loss-1.3098"

In [45]:
def predict_answer(question, options):
    formatted_question = f"question: {question} options: {' | '.join(options)}"
    prediction = model.predict(formatted_question)[0]
    return prediction if prediction in options else options[0]

In [29]:
# let's load the trained model for inferencing
model.load_model("t5","./outputs/simplet5-epoch-2-train-loss-0.1189-val-loss-0.1992", use_gpu=True)

In [30]:
test_data['source_text']

7900    question: question: Which frequency bands can ...
7901    question: question: Which type of neural netwo...
7902    question: question: In an infrastructure BSS, ...
7903    question: question: What is the concept of cel...
7904    question: question: What are the main control ...
                              ...                        
7995    question: question: What does the capacitor do...
7996    question: question: What is the modulation tec...
7997    question: question: What is the goal of vector...
7998    question: question: Which interface is not req...
7999    question: question: What is the purpose of ful...
Name: source_text, Length: 100, dtype: object

In [31]:
q_test = test_data['source_text'][7976]
q_option = test_data['options'][7976]
q_ans = test_data['target_text'][7976]

print("Question: ", q_test)
print('-'*50)
print("Options: " ,q_option )
print('-'*50)
print("Answer: ",q_ans)

Question:  question: question: What is the purpose of secondary CNPC (control and non-payload communications) links via satellite? options: To provide high-speed data communication between UAVs | To enhance reliability and robustness as a backup | To control UAVs flying within a controlled airspace | To provide secure authentication for CNPC links | To support mission-related communications for ground terminals
--------------------------------------------------
Options:  ['To provide high-speed data communication between UAVs', 'To enhance reliability and robustness as a backup', 'To control UAVs flying within a controlled airspace', 'To provide secure authentication for CNPC links', 'To support mission-related communications for ground terminals']
--------------------------------------------------
Answer:  To enhance reliability and robustness as a backup


In [32]:
predicted_ans = model.predict(q_test)[0]

print(predicted_ans)

To control UAVs flying within a controlled airspace


# Evaluation


In [34]:
df_test = pd.read_json('../input/dataset-haick/test.json')

In [35]:
questions = []
for ind in range(2000):
    questions.extend(df_test.loc[df_test[f'question {ind}'].notna(), f'question {ind}'].tolist())
df_test_copy = df_test.copy()
df_test_copy['question 0'] = questions
df_test_copy.rename(columns={'question 0': 'question'}, inplace=True)
df_test_copy.drop(columns=[f'question {ind}' for ind in range(1, 2000)], inplace=True)

In [36]:
df_test_copy2= df_test_copy.copy()

In [37]:
def split_options(row):
  options_list = row['options']
  return pd.Series({'A': options_list[0] if len(options_list) >= 1 else np.nan,
                   'B': options_list[1] if len(options_list) >= 2 else np.nan,
                   'C': options_list[2] if len(options_list) >= 3 else np.nan,
                   'D': options_list[3] if len(options_list) >= 4 else np.nan,
                   'E': options_list[3] if len(options_list) >= 5 else np.nan})

df_test_copy2 = df_test_copy2.apply(split_options, axis=1)

In [38]:
df_test_copy2.sample(1)

,A,B,C,D,E
1860,Orthogonal and shared resource allocation,Dynamic and static resource allocation,CDMA and FDMA resource allocation,Single and multi-carrier resource allocation,Single and multi-carrier resource allocation


In [39]:
concatenated_df = pd.concat([df_test_copy, df_test_copy2], axis=1)
concatenated_df.drop(columns = 'options')

,question,A,B,C,D,E
0,Which type of model is a neural network (NN)?,Linear regression,Nonparametric model,k-nearest neighbor model,Gaussian mixture model,Gaussian mixture model
1,Why do AV production applications call for unc...,To amplify defects in the transmitted data.,To match or improve upon existing workflows.,To propagate communication errors to the entir...,To post-process the transmitted data with filt...,To post-process the transmitted data with filt...
2,What is the purpose of energy efficiency metri...,To capture the energy consumed by the system u...,To capture the performance measured at the net...,To allow comparisons between different systems,All of the above,NaN
3,What are some application domains where blockc...,Cryptocurrencies,Smart grid,Connected vehicles,Internet of Things,Internet of Things
4,What are the requirements for relay UE selecti...,Selection based on the subscription owner of t...,Selection based on the QoS achievable by the E...,Selection based on the power consumption requi...,Selection based on the access technology used ...,Selection based on the access technology used ...
...,...,...,...,...,...,...
1995,What is a safety program? [IEEE C95.1],A program designed to ensure compliance with e...,A program to assess the likelihood and consequ...,A program to evaluate the spatial average of a...,A program to calculate the root-mean-square (r...,A program to calculate the root-mean-square (r...
1996,What is a base station?,A network element responsible for radio transm...,A type of telecommunication service that provi...,The time during which the base station is rece...,A value denoting unidirectional distribution t...,A value denoting unidirectional distribution t...
1997,What is the general concept behind deep learning?,Approximating continuous functions on compact ...,Detecting and classifying signals in wireless ...,Unfolding a predicted gradient descent mean,Encoding and decoding transmitted signals,Encoding and decoding transmitted signals
1998,What do metasurfaces transform incident waves ...,Desired reflected and transmitted waves,Randomly scattered waves,Uniform plane waves,Evanescent waves,Evanescent waves


In [40]:
df_testest = concatenated_df.copy()

In [41]:
df_testest['question'][1]

'Why do AV production applications call for uncompressed or slightly compressed data? [3GPP Release 17]'

In [42]:
predicted_ans = model.predict(df_testest['question'][1])
q_ans = df_testest['D'][1]

In [43]:
print(predicted_ans[0])
print(q_ans)

Why do AV production applications call for uncompressed or slightly compressed data?
To post-process the transmitted data with filters.


In [46]:
predicted_ans = predict_answer(df_testest['question'][1] , df_testest['options'][1])
q_ans = df_testest['D'][1]
print(predicted_ans)
print(q_ans)

To match or improve upon existing workflows.
To post-process the transmitted data with filters.


In [47]:
predictions = []
for index, row in df_testest.iterrows():
    question = row['question']
    options = row['options']
    predicted_answer = predict_answer(question , options)
    
    # Map predicted answer to numerical label
    if predicted_answer == row['A']:
        predictions.append(1)
    elif predicted_answer == row['B']:
        predictions.append(2)
    elif predicted_answer == row['C']:
        predictions.append(3)
    elif predicted_answer == row['D']:
        predictions.append(4)
    elif predicted_answer == row['E']:
        predictions.append(5)
    else:
        predictions.append(1)
    

In [48]:
df_submission = pd.read_csv('../input/dataset-haick/sample_submission.csv')

In [49]:
df_submission

,question,label
0,question 0,1
1,question 1,1
2,question 2,1
3,question 3,1
4,question 4,1
...,...,...
1995,question 1995,1
1996,question 1996,1
1997,question 1997,1
1998,question 1998,1


In [50]:
df_submission['label'] = predictions

In [51]:
df_submission

,question,label
0,question 0,3
1,question 1,2
2,question 2,4
3,question 3,1
4,question 4,1
...,...,...
1995,question 1995,1
1996,question 1996,3
1997,question 1997,3
1998,question 1998,1


In [52]:
df_submission.to_csv('submission1.csv', index=False)